In [15]:
from pydantic_ai import Agent
from pydantic_ai.models.anthropic import AnthropicModel
import requests
from pydantic import BaseModel, Field
from typing import Any
import json

In [18]:
# Pydantic models for FHIR metadata

class SearchParameter(BaseModel):
    """FHIR search parameter definition"""
    name: str
    type: str | None  # Can sometimes be None if special type
    documentation: str | None = None

class ResourceMetadata(BaseModel):
    """Metadata for a single FHIR resource type"""
    type: str
    profile: str | None = None
    interactions: list[str]  # ['read', 'search-type', 'create', etc.]
    search_params: list[SearchParameter]

class FHIRMetadata(BaseModel):
    """Complete FHIR server metadata response"""
    searchable_types: list[str] = Field(
        description="List of resource types that support search-type interaction"
    )
    resource_metadata: dict[str, ResourceMetadata] = Field(
        description="Full metadata for each resource type, keyed by type name"
    )
    fhir_version: str | None = None
    server_url: str

In [20]:
def fetch_searchable_resources(base_url: str = "https://r4.smarthealthit.org") -> FHIRMetadata:
    """
    Fetch searchable resource types from FHIR server metadata.

    Based on reference code that queries /metadata endpoint and filters
    resources with 'search-type' interaction capability.

    Args:
        base_url: FHIR server base URL (default: SMART Health IT R4 server)

    Returns:
        FHIRMetadata: Pydantic model containing searchable types and full resource metadata

    Raises:
        requests.RequestException: If network request fails
        ValueError: If response is invalid or missing required fields
    """
    # Query metadata endpoint
    metadata_url = f"{base_url}/metadata"

    try:
        response = requests.get(metadata_url, timeout=10)
        response.raise_for_status()
        capability_statement = response.json()
    except requests.RequestException as e:
        raise requests.RequestException(f"Failed to fetch metadata from {metadata_url}: {e}")

    # Validate response structure
    if not capability_statement.get('rest') or len(capability_statement['rest']) == 0:
        raise ValueError("Invalid CapabilityStatement: missing 'rest' array")

    if not capability_statement['rest'][0].get('resource'):
        raise ValueError("Invalid CapabilityStatement: missing 'resource' array in rest[0]")

    # Extract searchable resources (matching reference code logic)
    searchable_types: list[str] = []
    resource_metadata: dict[str, ResourceMetadata] = {}

    for resource in capability_statement['rest'][0]['resource']:
        resource_type = resource.get('type')
        if not resource_type:
            continue

        # Check if resource supports search-type interaction (reference line 74-79)
        interactions = resource.get('interaction', [])
        interaction_codes = [interaction.get('code') for interaction in interactions]

        if 'search-type' in interaction_codes:
            searchable_types.append(resource_type)

            # Parse search parameters and sort alphabetically (reference line 81-90)
            search_params: list[SearchParameter] = []
            for param in resource.get('searchParam', []):
                search_params.append(SearchParameter(
                    name=param.get('name'),
                    type=param.get('type'),
                    documentation=param.get('documentation')
                ))

            # Sort by name (matching reference code)
            search_params.sort(key=lambda p: p.name)

            # Create ResourceMetadata object
            resource_metadata[resource_type] = ResourceMetadata(
                type=resource_type,
                profile=resource.get('profile'),
                interactions=interaction_codes,
                search_params=search_params
            )

    return FHIRMetadata(
        searchable_types=searchable_types,
        resource_metadata=resource_metadata,
        fhir_version=capability_statement.get('fhirVersion'),
        server_url=base_url
    )

In [21]:
# Test the function
metadata = fetch_searchable_resources()

print(f"FHIR Version: {metadata.fhir_version}")
print(f"Server URL: {metadata.server_url}")
print(f"\nFound {len(metadata.searchable_types)} searchable resource types:")
print(metadata.searchable_types[:10])  # Show first 10

# Examine a specific resource
if 'Patient' in metadata.resource_metadata:
    patient_meta = metadata.resource_metadata['Patient']
    print(f"\nPatient resource:")
    print(f"  Interactions: {patient_meta.interactions}")
    print(f"  Number of search parameters: {len(patient_meta.search_params)}")
    print(f"  First 5 search parameters: {[p.name for p in patient_meta.search_params[:5]]}")

FHIR Version: 4.0.0
Server URL: https://r4.smarthealthit.org

Found 146 searchable resource types:
['Account', 'ActivityDefinition', 'AdverseEvent', 'AllergyIntolerance', 'Appointment', 'AppointmentResponse', 'AuditEvent', 'Basic', 'Binary', 'BiologicallyDerivedProduct']

Patient resource:
  Interactions: ['read', 'vread', 'update', 'patch', 'delete', 'history-instance', 'history-type', 'create', 'search-type']
  Number of search parameters: 25
  First 5 search parameters: ['_id', '_language', 'active', 'address', 'address-city']


In [4]:
model = AnthropicModel('claude-opus-4-5')


UserError: Set the `ANTHROPIC_API_KEY` environment variable or pass it via `AnthropicProvider(api_key=...)`to use the Anthropic provider.